In [1]:
import pandas as pd
import mlflow
import sys

# Add the src directory to the Python path so this notebook can import our custom modules.
# We use the absolute path that exists inside the Jupyter container.
sys.path.append('/home/jovyan/src')

# Import the superior, class-based decision engine
from MCDA_model import HRES_Decision_Engine

# --- Load Resources ---
# Load the comprehensive dataset that was generated by our simulation engine
DATA_PATH = "/home/jovyan/src/hres_configurations_superior.csv"
hres_df = pd.read_csv(DATA_PATH)

# Instantiate our decision engine with the loaded data
try:
    decision_engine = HRES_Decision_Engine(hres_df)
    print(f"✅ HRES Decision Engine initialized with {len(hres_df)} configurations.")
except Exception as e:
    print(f"❌ Error initializing HRES Decision Engine: {e}")

# --- MLflow Setup ---
# Set the name for our experiment. If it doesn't exist, MLflow will create it.
EXPERIMENT_NAME = "HRES_Decision_Scenarios_Superior"
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"✅ MLflow experiment '{EXPERIMENT_NAME}' is ready.")

2025/09/21 15:31:52 INFO mlflow.tracking.fluent: Experiment with name 'HRES_Decision_Scenarios_Superior' does not exist. Creating a new experiment.


✅ HRES Decision Engine initialized with 625 configurations.
✅ MLflow experiment 'HRES_Decision_Scenarios_Superior' is ready.


In [2]:
# --- Cell 2 (Corrected) ---
scenarios = {
    "University_High_ESG": {
        "scenario_name": "University_Campus",
        "annual_demand_kwh": 3000000, # Add this
        "user_grid_dependency_pct": 15,
        "esg_weights": {"environmental": 0.5, "social": 0.4, "governance": 0.1, "cost": 0.0}
    },
    "Office_Cost_Focused": {
        "scenario_name": "Small_Office",
        "annual_demand_kwh": 250000, # Add this
        "user_grid_dependency_pct": 30,
        "esg_weights": {"environmental": 0.1, "social": 0.1, "governance": 0.0, "cost": 0.8}
    }
    # You can add more scenarios here
}

for run_name, params in scenarios.items():
    with mlflow.start_run(run_name=run_name) as run:
        mlflow.log_params(params)
        
        # THIS IS THE FIX: Add the missing annual_demand_kwh argument
        best_solution, status, feasible_df, sorted_df = decision_engine.run_full_pipeline(
            params["scenario_name"],
            params["annual_demand_kwh"], # The missing argument
            params["user_grid_dependency_pct"],
            params["esg_weights"]
        )
        
        mlflow.set_tag("status", status)

        if best_solution is not None:
            metrics_to_log = best_solution.drop(['scenario_name', 'esg_category', 'annual_demand_kwh']).to_dict()
            mlflow.log_metrics(metrics_to_log)

print("\n🎉 All scenarios have been run and logged to MLflow.")

🏃 View run University_High_ESG at: http://mlflow:5000/#/experiments/1/runs/7b22ceeb8310406391dab72a5a72ee1c
🧪 View experiment at: http://mlflow:5000/#/experiments/1


KeyError: "['env_water_savings_m3_yr', 'env_waste_factor_pct', 'env_degradation_rate', 'soc_energy_resilience_hrs', 'soc_grid_strain_reduction_pct', 'soc_community_investment_usd', 'soc_noise_level_impact_score', 'gov_regulatory_compliance_score', 'gov_stakeholder_reporting_score', 'gov_operational_risk_score'] not in index"

In [ ]:
# THIS CELL IS FOR DEMONSTRATING THE MODEL REGISTRY REQUIREMENT
import mlflow.pyfunc

MODEL_NAME = "HRES-Recommender-Model-Packaged"

# We create a simple wrapper for the class to make it compatible with mlflow.pyfunc
class HRES_Engine_Wrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.engine = None
        
    def load_context(self, context):
        DATA_PATH = "/app/src/hres_configurations_superior.csv"
        hres_df = pd.read_csv(DATA_PATH)
        self.engine = HRES_Decision_Engine(hres_df)

    def predict(self, context, model_input):
        results = []
        for _, row in model_input.iterrows():
            solution, status = self.engine.run_full_pipeline(
                row['scenario_name'],
                row['user_grid_dependency_pct'],
                eval(row['esg_weights'])
            )
            if solution is not None:
                res_dict = solution.to_dict()
                res_dict['status'] = status
                results.append(res_dict)
            else:
                results.append({'status': status, 'recommendation': None})
        return pd.DataFrame(results)

# Log and register the wrapped model
with mlflow.start_run(run_name="Package and Register Logic") as run:
    input_example = pd.DataFrame([{
        "scenario_name": "University_Campus",
        "user_grid_dependency_pct": 15.0,
        "esg_weights": str({"environmental": 0.5, "social": 0.4, "governance": 0.1, "cost": 0.0})
    }])
    
    mlflow.pyfunc.log_model(
        artifact_path="model_logic",
        python_model=HRES_Engine_Wrapper(),
        input_example=input_example,
        registered_model_name=MODEL_NAME
    )
    
print(f"✅ Decision logic has been packaged and registered as model '{MODEL_NAME}' in the MLflow Model Registry.")